<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

 # [Apprentissage en Grande Dimension](https://github.com/wikistat/High-Dimensional-Learning ) : [Reconnaissance d'Activité Humaine](https://github.com/wikistat/High-Dimensional-Learning/tree/master/HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>  Seconde partie:  apprentissage (profond) des signaux bruts  avec <a href="https://keras.io/"><img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" style="max-width: 100px; display: inline" alt="Keras"/></a>

Ce notebook présente la partie prediction de l'activité. Pour l'exploration, se référer au calepin afférent.

#   Introduction
##   Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles ont été acquises, décrites et analysées par [Anguita et al. (2013)](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-84.pdf). Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z, soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.

Les méthodes d'apprentissage sont appliquées sur ces données brutes, sans calculs préliminaires de caractéristiques (*features*).

##  Objectif
Cette deuxième étape s'intéresse aux données brutes. Est-il possible d'économiser le travail préliminaire de définition des variables métier en utilisant, par exemple, un algorithme d'apprentissage profond sur les données brutes ou leur transformation en ondelettes ?

**Objectif** Faire aussi bien (96% de bien classés) qu'avec les variables métier.

##  Méthodes abordées : 
**Attention l'accès à un environnement *GPU* est très vivement conseillé voire indispensable.**
- Modélisation, prévision de l'échantillon test par
   - Régression logistique (`Scikit-learn`) sur signaux "applatis"
   - Apprentissage profond en utilisant `Keras` 
       - MLP sur signaux "applatis"
       - MLP sur signaux mutlidimensionels
       - 1D Convolution
       - 2D Convolution
   - Validation Monte Carlo pour choisir le modèle
   
   
- ** Extensions possibles  à ce travail: **

    - Application des méthodes d'apprentissage classiques et de l'apprentissage profond sur les coefficients des décompositions des signaux en ondelettes
    - Optimisation des paramètres des différentes méthodes.
    - Améliorer l'architecture des réseaux ? 


#  Mise en place
## Librairies et initialisation

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import copy
import random
import itertools

#Utils Sklearn
import sklearn.linear_model as lm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV,train_test_split

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

# DEEP LEARING
import tensorflow as tf
np.random.seed(42)
tf.set_random_seed(42)

# for reproducibility
# https://github.com/fchollet/keras/issues/2280
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras.models as km 
import keras.layers as kl 
import keras.layers.core as klc

##  Prise en charge des données
### Sources

Les données sont celles originales du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent être téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Elles contiennent deux jeux de dimensions différentes, chacun partagé en apprentissage et test.

1. Multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(N, 128, 9)$
2. Unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128*9 = 1152 variables de *dimensions* $(N, 1152)*
        
Deux objets différents sont construits pour définir la variable $Y$ réponse car les librairies `Scikit-learn` et `Keras` prennent en compte des structures différentes: 
    
1. `Scikit-Learn`  Un vecteur de dimension $(N, 1)$ avec, pour chaque individu le numéro du label de l'activité de 0 à 5.
2. `Keras` Une matrice de dimension $(N, 6)$ des indicatrices (0 ou 1) des modalités de $Y$.

### Lecture des données

In [ ]:
# attention: adapter si besoin le chemin d'accès aux données

DATADIR_UCI = '.'
SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]

def my_read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signal(data_dir, subset, signal):
    filename = f'{data_dir}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    x = my_read_csv(filename).as_matrix()
    return x 

def load_signals(data_dir, subset, flatten = False):
    signals_data = []
    for signal in SIGNALS:
        signals_data.append(load_signal(data_dir, subset, signal)) 
    
    if flatten :
        X = np.hstack(signals_data)
    else:
        X = np.transpose(signals_data, (1, 2, 0))
        
    return X 

def load_y(data_dir, subset, dummies = False):
    filename = f'{data_dir}/{subset}/y_{subset}.txt'
    y = my_read_csv(filename)[0]
    
    
    if dummies:
        Y = pd.get_dummies(y).as_matrix()
    else:
        Y = y.as_matrix()
    
    return Y

Vérification des dimensions

In [ ]:
#Multidimensional Data
X_train, X_test = load_signals(DATADIR_UCI, 'train'), load_signals(DATADIR_UCI, 'test')
# Flattened Data
X_train_flatten, X_test_flatten = load_signals(DATADIR_UCI, 'train', flatten=True), load_signals(DATADIR_UCI, 'test', flatten=True)

# Label Y
Y_train_label, Y_test_label = load_y(DATADIR_UCI, 'train', dummies = False), load_y(DATADIR_UCI, 'test', dummies = False)
#Dummies Y (For Keras)
Y_train_dummies, Y_test_dummies = load_y(DATADIR_UCI, 'train', dummies = True), load_y(DATADIR_UCI, 'test', dummies = True)

N_train = X_train.shape[0]
N_test = X_test.shape[0]

In [ ]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse (scikit-learn) : " + str(Y_train_label.shape))
print("Matrice réponse(Keras) : " + str(Y_train_dummies.shape))

#### Utilitaires

In [ ]:
LABELS = ["WALKING","WALKING UPSTAIRS","WALKING DOWNSTAIRS","SITTING","STANDING","LAYING"]
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def my_confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

# Apprentissage des signaux uni-dimensionels

La base d'apprentissage est de dimension (`N_train`, 1152)

##  Régression Logistique

La Régression Logistique est une des méthodes conduisant aux meilleurs résultats sur les variables métier.

In [ ]:
t_start = time.time()
model_lr = lm.LogisticRegression(verbose=1)
model_lr.fit(X_train_flatten, Y_train_label)
t_end = time.time()
t_learning = t_end-t_start
score = model_lr.score(X_test_flatten, Y_test_label)
print("Score With Logistic Regression on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
lr_prediction_label = model_lr.predict(X_test_flatten)
metadata_svc = {"time_learning" : t_learning, "score" : score}
pd.DataFrame(confusion_matrix(lr_prediction_label, Y_test_label), index = LABELS, columns=LABELS)

**Q** Que dire de la performance?

## Perceptron multicouche

Un réseau de neurones classique est appris sur les données au même format que précédemment.

**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [ ]:
epochs = 10
batch_size = 32
n_hidden = 32

n_features = X_train_flatten.shape[1]
n_classes=6


model_base_mlp_u =km.Sequential()
model_base_mlp_u.add(kl.Dense(n_hidden, input_shape=(n_features,),  activation = "relu"))
model_base_mlp_u.add(kl.Dropout(0.5))
model_base_mlp_u.add(kl.Dense(n_classes, activation='softmax'))
model_base_mlp_u.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model_base_mlp_u.summary()

In [ ]:
t_start = time.time()
model_base_mlp_u.fit(X_train_flatten,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_flatten, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp_u.evaluate(X_test_flatten, Y_test_dummies)[1] 
print("Score With Simple MLP on Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp_u = {"time_learning" : t_learning, "score" : score}
base_mlp_u_prediction = model_base_mlp_u.predict(X_test_flatten)

my_confusion_matrix(Y_test_dummies, base_mlp_u_prediction)

** Q ** : Que conclure sur ces résultats en terme de performance, de temps d'apprentissage? Comparer avec la regression logistique?

** Exercice ** : Quelle est l'influence de l'ajout de nouvelle couche? Supression du Dropout? ...

# Apprentissage des signaux multidimensionnels
Les différents signaux ne sont pas concaténés en un seul signal mais pris en compte parallèlement.

## Perceptron multichouche
**Q** Expliciter les choix des paramètres et donc la structure du réseau.

In [ ]:
n_hidden = 50

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

model_base_mlp =km.Sequential()
model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))

model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_mlp.summary()

In [ ]:
t_start = time.time()
model_base_mlp.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_mlp.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Simple MLP on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_mlp = {"time_learning" : t_learning, "score" : score}
base_mlp_prediction = model_base_mlp.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_mlp_prediction)

## Réseau avec couche convolutionelle 1D (*ConvNet*)

L'idée pertinente avec ces données est évidemment d'identifier le problème lié au déphasage des signaux. L'utilisation d'une couche convolutionnelle introduit une propriété d'invariance par translation. Les caractéristiques ou *features* sortant de cette couche acquièrent donc ainsi de bonnes propriétés avant d'être dirigées vers des couches techniques intermédiaires (`MaxPooling, Flatten`) et une dernière couche de sortie qui effectue la discrimination à partir des caractéristiques.

**Q.** Remarquer le nombre de paramètres à estimer, le comparer avec celui du perceptron précédent et comprendre comment la convolution 1D agit sur les signaux (regarder en particulier la forme de la sortie de chaque couche du réseau). 

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

#else:
model_base_conv_1D =km.Sequential()
model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
model_base_conv_1D.add(kl.Flatten())
model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_conv_1D.summary()

In [ ]:
t_start = time.time()
model_base_conv_1D.fit(X_train,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_1D.evaluate(X_test, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_1D_prediction = model_base_conv_1D.predict(X_test)

my_confusion_matrix(Y_test_dummies, base_conv_1D_prediction)

## Réseau avec couche convolutionelle 2D (*ConvNet*)

**Q.** Remarquer le nombre de paramètres à estimer, le comparer avec celui du réseau précédent et comprendre comment la convolution 2D agit sur les signaux (regarder en particulier la forme de la sortie de chaque couche du réseau). 

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

X_train_conv = X_train.reshape(N_train, timesteps, input_dim, 1)
X_test_conv = X_test.reshape(N_test, timesteps, input_dim, 1)

#else:
model_base_conv_2D =km.Sequential()
model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
model_base_conv_2D.add(kl.Flatten())
model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model_base_conv_2D.summary()

In [ ]:
t_start = time.time()
model_base_conv_2D.fit(X_train_conv,  Y_train_dummies, batch_size=batch_size, validation_data=(X_test_conv, Y_test_dummies), epochs=epochs)
t_end = time.time()
t_learning = t_end-t_start

score = model_base_conv_2D.evaluate(X_test_conv, Y_test_dummies)[1] 
print("Score With Conv on Multidimensional Inertial Signals = %.2f, Learning time = %.2f secondes" %(score*100, t_learning) )
metadata_conv = {"time_learning" : t_learning, "score" : score}
base_conv_2D_prediction = model_base_conv_2D.predict(X_test_conv)

my_confusion_matrix(Y_test_dummies, base_conv_2D_prediction)



**Attention au sur-apprentissage** A force de rechercher la meilleure architecture en minimisant l'erreur sur l'échantillon test, celle finalement trouvée peut y être très adaptée réduisant ainsi la capacité de généralisation. Il serait prudent de multiplier le découpage de l'échantillon par validation croisée *Monte Carlo*.

# Implémentation de la Validation Croisée Monte Carlo

**Objectif** :  trouver la meilleure architecture.

In [ ]:
X=np.copy(np.concatenate((X_train, X_test), axis=0))
Y=np.copy(np.concatenate((Y_train_dummies, Y_test_dummies), axis=0))
Y.shape

In [ ]:
epochs = 1
#Il faudrait en prendre plus ..
batch_size = 32
n_hidden = 32
n_classes = 6

In [ ]:
N_MC=10

score=np.empty([N_MC,3])


for k in range(N_MC):
    print("\n \n *****************",k,"***************** \n")
    X_train_MC,X_test_MC,Y_train_dummies_MC,Y_test_dummies_MC=train_test_split(X,Y,test_size=0.2)
    N_train_MC = X_train_MC.shape[0]
    N_test_MC = X_test_MC.shape[0]
    
    timesteps = len(X_train_MC[0])
    input_dim = len(X_train_MC[0][0])
    
    X_train_conv_MC = X_train_MC.reshape(N_train_MC, timesteps, input_dim, 1)
    X_test_conv_MC = X_test_MC.reshape(N_test_MC, timesteps, input_dim, 1)
    
    
    # définition des modèles 
    model_base_mlp =km.Sequential()
    model_base_mlp.add(kl.Dense(n_hidden, input_shape=(timesteps, input_dim),  activation = "relu"))
    model_base_mlp.add(kl.Reshape((timesteps*n_hidden,) , input_shape= (timesteps, n_hidden)  ))
    model_base_mlp.add(kl.Dense(n_classes, activation='softmax'))
    model_base_mlp.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model_base_conv_1D =km.Sequential()
    model_base_conv_1D.add(kl.Conv1D(32, 9, activation='relu', input_shape=(timesteps, input_dim)))
    model_base_conv_1D.add(kl.MaxPooling1D(pool_size=3))
    model_base_conv_1D.add(kl.Flatten())
    model_base_conv_1D.add(kl.Dense(n_classes, activation='softmax'))
    model_base_conv_1D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model_base_conv_2D =km.Sequential()
    model_base_conv_2D.add(kl.Conv2D(32, (3, 9), activation='relu', input_shape=(timesteps, input_dim, 1)))
    model_base_conv_2D.add(kl.MaxPooling2D(pool_size=(2, 1)))
    model_base_conv_2D.add(kl.Flatten())
    model_base_conv_2D.add(kl.Dense(n_classes, activation='softmax'))
    model_base_conv_2D.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
   
    print("\n **** MLP **** \n")
    model_base_mlp.fit(X_train_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_MC, Y_test_dummies_MC), epochs=epochs)   

    print("\n **** conv 1D **** \n")
    model_base_conv_1D.fit(X_train_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_MC, Y_test_dummies_MC), epochs=epochs)
   
    print("\n **** conv 2D **** \n")
    model_base_conv_2D.fit(X_train_conv_MC,  Y_train_dummies_MC, batch_size=batch_size, validation_data=(X_test_conv_MC, Y_test_dummies_MC), epochs=epochs)
    
    score_mlp=model_base_mlp.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    #score_lstm=model_base_lstm.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    score_conv_1D=model_base_conv_1D.evaluate(X_test_MC, Y_test_dummies_MC)[1]
    score_conv_2D=model_base_conv_2D.evaluate(X_test_conv_MC, Y_test_dummies_MC)[1]
    s=[score_mlp,score_conv_1D,score_conv_2D]
    score[k,:]=s
    
final_scores=np.apply_along_axis(np.mean,0,score)*100
print("\n",score)
print(final_scores)

In [ ]:
print("MLP     :",round(final_scores[0],2),"% \n")
print("Conv 1D :",round(final_scores[1],2),"% \n")
print("Conv 2D :",round(final_scores[2],2),"% \n")

**Q.** Quelle méthode est la plus performante ? Commenter ces résultats par rapport à ceux qui ont été obtenus sur les données "métier". 